# A09 - Bagging

Gonzalo Cano Padilla

**INSTRUCCIONES DE LA ACTIVIDAD**
* En este caso como nuestro dataset es 'Default' usaremos arboles de decisión de clasificación
* Primero una regresión logística
* Luego Bagging (bootstrap + aggregatin)
* Sacar Accuracy de ambas
* Compararlo
* Conclusiones

**PASOS PARA RESOLVERLO**
* Leer el dataset
* Separar X y Y, tratar las variables categoricas
* Hacer la regresión
* Hacer el bootsrap, donde entran X y Y, luego hacer el resample con 5000 remuestreos
* Hacer el aggregating, aqui solo entran las X -> luego seleccionar 2 columnas random -> identificar columnas -> np.random.choice n=2 sin remplazo

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

df_def = pd.read_csv('Default.csv')
df_def

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880
...,...,...,...,...
9995,No,No,711.555020,52992.378910
9996,No,No,757.962918,19660.721770
9997,No,No,845.411989,58636.156980
9998,No,No,1569.009053,36669.112360


---

## Regresión Logística

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

# Variables
y = (df_def['default'] == 'Yes').astype(int).values
X = np.column_stack([
    df_def['balance'].values,
    df_def['income'].values,
    (df_def['student'] == 'Yes').astype(int).values
])

# Regresión logística
lr = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X, y)
y_pred = lr.predict(X)
accuracy_lr = accuracy_score(y, y_pred)
print(f'Logistic Regression Accuracy: {accuracy_lr:.2%}')

Logistic Regression Accuracy: 97.32%


In [6]:
# Bagging
from sklearn.utils import resample
from scipy.stats import mode

n_sim = 5000
preds = []

for b in range(n_sim):
    X_boot,y_boot = resample(X, y, replace=True, n_samples=5000)
    cols = np.random.choice(X_boot.shape[1], size=2, replace=False)
    X_boot_agg = X_boot[:, cols]
    tree = DecisionTreeClassifier().fit(X_boot_agg, y_boot)
    pred_boot = tree.predict(X[:,cols])
    preds.append(pred_boot)

preds = np.array(preds)
pred_final = mode(preds, axis=0, keepdims=False).mode

acc_bagging = accuracy_score(y, pred_final)
print(f'Bagging Accuracy: {acc_bagging:.2%}')

Bagging Accuracy: 98.46%


---
## Interpretaciones

El modelo de bagging obtuvo un mejor accuracy que la regresión logística, esto se debe a que con el bagging hicimos muchos arboles y cada arbol tuvo errores pero al final entre todos los arboles se "voto" para que los errores individuales de cada arbol desaparezcan y asi dejando solo las relaciones verdaderas se quedan.

En la regresión logística el accuracy fue menor ya que no captura las relaciones no lineales entre las variables entonces no captura la complejidad del patrón y hace que se equivoque mas es por eso que el bagging tuvo mejor accuracy ya que los arboles tienen más capacidad para las relaciones no lineales y el baggin reduce la varianza de los arboles.